In [ ]:
# pip install websockets pylsl

In [5]:
import asyncio
import websockets
import json
from pylsl import StreamInfo, StreamOutlet

# Create an LSL stream
info = StreamInfo('WebSocketLSL', 'Markers', 1, 0, 'string', 'myuid12345')
outlet = StreamOutlet(info)

# Accept a variable number of arguments so it works with either 1 or 2 parameters.
async def handle_connection(websocket, *args):
    # If a path is provided, use it; otherwise, default to None
    path = args[0] if args else None
    print(f"Client connected. Path: {path}")

    try:
        async for message in websocket:
            print(f"Received WebSocket message: {message}")

            try:
                data = json.loads(message)
                event_marker = data.get("event", "UNKNOWN_EVENT")
                timestamp = data.get("timestamp", "No timestamp")

                # Push the event to LSL
                outlet.push_sample([event_marker])
                print(f"Sent to LSL: {event_marker} at {timestamp}")

                # Send acknowledgment back to the client
                await websocket.send(f"LSL Received: {event_marker}")

            except json.JSONDecodeError:
                print("Error: Received non-JSON message")

    except websockets.exceptions.ConnectionClosed:
        print("Client disconnected")

async def main():
    print("Starting WebSocket Server on ws://localhost:8765")
    async with websockets.serve(handle_connection, "localhost", 8765):
        await asyncio.Future()  # Keep the server running

# Use the current event loop or start a new one if needed.
try:
    loop = asyncio.get_running_loop()
    loop.create_task(main())
    print("WebSocket server running in background...")
except RuntimeError:
    asyncio.run(main())

WebSocket server running in background...


Starting WebSocket Server on ws://localhost:8765
